In [1]:
!pip --quiet install pandas-datareader
!pip --quiet install yfinance

     |████████████████████████████████| 6.4 MB 13.3 MB/s 


In [2]:
import pandas_datareader.data as reader
import pandas as pd
import datetime as dt
import statsmodels.api as sm
import yfinance as yf

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
df_esg=pd.read_csv('df_monthly_adobe.csv')
df_esg=df_esg.set_index('Date')
df_esg=df_esg[:59]

FileNotFoundError: ignored

In [ ]:
df_esg.tail()

In [ ]:
# df_esg=df_esg['AvgTone'].pct_change()

In [ ]:
end = dt.date(2021,12,1)
start = dt.date(2016,12,1)
funds = ['ADBE']

In [ ]:
fundsret = pd.DataFrame()
for stock in funds:
    fundsret[stock] = yf.download(stock, data_source="yahoo", start=start,end=end)['Adj Close'].pct_change()
fundsret.head()


In [ ]:
fundsret_mtl = fundsret.resample('M').agg(lambda x:(x+1).prod()-1)
fundsret_mtl = fundsret_mtl[1:]
fundsret_mtl

In [ ]:
factors = reader.DataReader('F-F_Research_Data_Factors','famafrench',start,end)[0]
# factors = factors[1:]


In [ ]:
fundsret_mtl.shape

In [ ]:
factors.shape

In [ ]:
df_esg.shape

In [ ]:
fundsret_mtl.index = factors.index

In [ ]:
merge = pd.merge(fundsret_mtl,factors, on = 'Date')

In [ ]:
merge['ADBE-RF'] = merge.ADBE - merge.RF


In [ ]:
merge.tail()

In [ ]:
df_esg.index=merge.index

In [ ]:
merge_2 = pd.merge(merge,df_esg, on = 'Date')

In [ ]:
merge_2.tail()

In [ ]:
y = merge_2['ADBE-RF']
X = merge_2[['Mkt-RF','SMB','HML','AvgTone']]

X_sm = sm.add_constant(X)

In [ ]:
model = sm.OLS(y,X_sm)
results = model.fit()
results.summary()

In [ ]:
!pip install --quiet pingouin
import pingouin as pg

In [ ]:
import numpy as np

In [ ]:
pg.linear_regression(np.log(df[['AvgTone']]), np.log(df['Adj Close']))

In [ ]:
merge_2

In [ ]:
import yfinance as yf

ADBE = yf.Ticker("ADBE")
data = yf.download("ADBE", start="2016-12-01", end="2021-10-31")[['Adj Close']]
data=data.resample('M').mean()

In [ ]:
data.index=df_esg.index

In [ ]:
df = pd.merge(data,df_esg, on = 'Date')

In [ ]:
import matplotlib.pyplot as plt
df_esg["AvgTone"].plot(figsize=(12,8),fontsize=10,color='r')
plt.ylabel('Close Price', fontsize=12)
data["Adj Close"].plot(figsize=(12,8),fontsize=10,secondary_y=True)

plt.xlabel('Date', fontsize=12)
# df["MA 20"][20:].plot(figsize=(10,5),color="Red")
plt.show()